In [2]:
import numpy as np
import requests
from selenium import webdriver

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select
import csv
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
pages = []

In [3]:
# Open Browser of all players 
driver = webdriver.Chrome()  


url = "https://www.nba.com/players"
driver.get(url)

In [4]:
# Manually change page through all 12 windows. Run this Block each time
page_source = driver.page_source

# Parse the HTML content
soup = BeautifulSoup(page_source, "html.parser")
playerList = soup.find_all(class_="primary text RosterRow_primaryCol__1lto4")
pages.append(playerList)

In [6]:
# Initialize playerURLS to start
playerUrls = []

In [7]:
# Parse the HTML to get URLS
for playerList in pages:
    
    for player in playerList:
        soup = BeautifulSoup(str(player), 'html.parser')
        
        # Find the <a> tag with the specified class
        a_tag = soup.find('a', class_='Anchor_anchor__cSc3P')

        if a_tag == None: continue
        # Extract the value of the 'href' attribute
        url = a_tag['href']
        url_parts = url.split("/")
    
        # Join the first three parts back together
        desired_url = "/".join(url_parts[:3])
        
        playerUrls.append(desired_url)

In [3]:
# Initialize the Final dict
playerPasses = {}

In [9]:
playerPasses

{}

In [4]:
#Stop warnings from showing up (they come up in pass pages
import warnings


warnings.simplefilter(action='ignore', category=FutureWarning)


In [6]:
playerUrls = ['/player/204001']

In [7]:
#Check how long the array is
len(playerUrls)

1

In [10]:
#Go through list of player links. Donwload the data and hold in Dictionary

for playerLink in playerUrls[331:]:

    url = f"https://www.nba.com/stats{playerLink}/passes-dash"

    # Set up Selenium webdriver (you need to have appropriate webdriver installed, e.g., chromedriver)
    driver = webdriver.Chrome()
    
    # Load the webpage
    driver.get(url)
    
    # Wait for the page to fully load (adjust the time as needed)
    driver.implicitly_wait(15)  # Wait for up to 15 seconds
    
    # Get the page source after dynamic content has been loaded
    page_source = driver.page_source
    
    # Close the webdriver
    driver.quit()
    
    # Parse the page source with BeautifulSoup
    soup = BeautifulSoup(page_source, "html.parser")
    
    
    # Find the table
    tables = soup.find_all(class_ = "Crom_table__p1iZz")
    
    

    summary = soup.find(class_ = "PlayerSummary_mainInnerBio__JQkoj")

    if summary == None: 
        print(f'ERROR at : {url}')
        continue
    
    names_array = [p.get_text(strip=True) for p in summary]
    name = f'{names_array[-2]}{names_array[-1]}'

    if len(tables) < 2:
        print(f'No data for {name}')
        continue
        
    
    passIn = tables[0]
    passOut = tables[1]

    summary = soup.find(class_ = "PlayerSummary_mainInnerBio__JQkoj")
    names_array = [p.get_text(strip=True) for p in summary]
    name = f'{names_array[-2]}{names_array[-1]}'
    
    # Convert the table to a DataFrame
    df_in = pd.read_html(str(passIn))[0]  # Assuming the table you want is the first one
    df_in.columns = df_in.columns.droplevel()
    freq_in = df_in[[df_in.columns[0],df_in.columns[2]]]
    
    
    
    df_out = pd.read_html(str(passOut))[0]  # Assuming the table you want is the first one
    df_out.columns = df_out.columns.droplevel()
    freq_out = df_out[[df_out.columns[0],df_out.columns[2]]]
    
    
    playerPasses[name] = (freq_in,freq_out)

In [19]:
len(playerPasses.keys())

560

## what do these pages have?
 df_in.columns = 'Pass To', 'Team', 'FREQ M%', 'PASS', 'AST', 'FGM', 'FGA', 'FG%',
       '2FGM', '2FGA', '2FG%', '3FGM', '3FGA', '3FG%', 'Unnamed: 14_level_1',
       'Unnamed: 15_level_1', 'Unnamed: 16_level_1', 'Unnamed: 17_level_1',
       'Unnamed: 18_level_1'

In [39]:
playerPasses.keys()

dict_keys(['PreciousAchiuwa', 'BamAdebayo', 'SantiAldama', 'GraysonAllen', 'JoseAlvarado', 'ThanasisAntetokounmpo', 'DeniAvdija', 'DeandreAyton', 'UdokaAzubuike', 'IbouBadji', 'LaMeloBall', 'MoBamba', 'PaoloBanchero', 'HarrisonBarnes', 'ScottieBarnes', 'CharlesBassey', 'KeitaBates-Diop', 'DariusBazley', 'BradleyBeal', 'MarJonBeauchamp', 'JulesBernard', 'DavisBertans', 'PatrickBeverley', 'OnuralpBitim', 'BismackBiyombo', 'LeakyBlack', 'BogdanBogdanovic', 'BolBol', 'BrandonBoston Jr.', 'ChrisBoucher', 'JamesBouknight', 'JamareeBouyea', 'MalakiBranham', 'ChristianBraun', 'MilesBridges', 'OshaeBrissett', 'IzaiahBrockington', 'ArmoniBrooks', 'KendallBrown', 'GregBrown III', 'CharlieBrown Jr.', 'ThomasBryant', 'ReggieBullock Jr.', 'JimmyButler', 'KentaviousCaldwell-Pope', 'ClintCapela', 'WendellCarter Jr.', 'AlexCaruso', 'GogaBitadze', 'AnthonyBlack', 'BuddyBoeheim', 'MarquesBolden', 'JustinChampagnie', 'MaxChristie', 'SidyCissoko', 'BrandonClarke', 'JohnCollins', 'MikeConley', 'BilalCouliba

In [2]:
##READ IN PLAYER INDEXES:
def strip_bracket1(word):
    return word.strip('[')
def strip_bracket2(word):
    return word.strip(']')
# Open the CSV file
with open('playerIndexesNBA.csv', newline='') as csvfile:
    # Create a CSV reader object
    csv_reader = csv.reader(csvfile)
    
    # Initialize an empty list to hold the data
    playerUrls = []
    
    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Append each row to the data_array
        playerUrls.append(row[0])

playerUrls = np.array(playerUrls)

In [3]:
playerUrls

array(['/player/1630173', '/player/203500', '/player/1628389',
       '/player/1630534', '/player/1630583', '/player/1629638',
       '/player/1628960', '/player/1628386', '/player/1630631',
       '/player/203937', '/player/203507', '/player/203648',
       '/player/1630175', '/player/1628384', '/player/1627853',
       '/player/1630166', '/player/1629028', '/player/1628962',
       '/player/1630641', '/player/1628963', '/player/1641735',
       '/player/1631116', '/player/1630163', '/player/1628366',
       '/player/1628964', '/player/1631094', '/player/1630217',
       '/player/1630625', '/player/1631230', '/player/203084',
       '/player/1630567', '/player/1629628', '/player/1629646',
       '/player/1641734', '/player/1628966', '/player/201587',
       '/player/1629647', '/player/203078', '/player/1627736',
       '/player/1630699', '/player/1631262', '/player/202722',
       '/player/201976', '/player/1630180', '/player/1629048',
       '/player/1641931', '/player/202687', '/pla

In [20]:
data = playerPasses
csv_file = 'playerPassingPercent.csv'

# Write the dictionary to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=data.keys())
    writer.writeheader()
    for row in zip(*data.values()):
        writer.writerow(dict(zip(data.keys(), row)))

In [45]:
playerUrls = np.array(playerUrls)

In [49]:
csv_file = 'playerIndexesNBA.csv'
array = playerUrls
np.savetxt(csv_file, array, delimiter=',', fmt='%s')


In [1]:
playerUrls

NameError: name 'playerUrls' is not defined

In [12]:
url = "https://www.nba.com/stats/player/204001/passes-dash"


# Set up Selenium webdriver (you need to have appropriate webdriver installed, e.g., chromedriver)
driver = webdriver.Chrome()

# Load the webpage
driver.get(url)

# Wait for the page to fully load (adjust the time as needed)
driver.implicitly_wait(15)  # Wait for up to 15 seconds

# Get the page source after dynamic content has been loaded
page_source = driver.page_source

# Close the webdriver
driver.quit()

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, "html.parser")


# Find the table
tables = soup.find_all(class_ = "Crom_table__p1iZz")



summary = soup.find(class_ = "PlayerSummary_mainInnerBio__JQkoj")


names_array = [p.get_text(strip=True) for p in summary]
name = f'{names_array[-2]}{names_array[-1]}'


    

passIn = tables[0]
passOut = tables[1]

summary = soup.find(class_ = "PlayerSummary_mainInnerBio__JQkoj")
names_array = [p.get_text(strip=True) for p in summary]
name = f'{names_array[-2]}{names_array[-1]}'

# Convert the table to a DataFrame
df_in = pd.read_html(str(passIn))[0]  # Assuming the table you want is the first one
df_in.columns = df_in.columns.droplevel()
freq_in = df_in[[df_in.columns[0],df_in.columns[2]]]



df_out = pd.read_html(str(passOut))[0]  # Assuming the table you want is the first one
df_out.columns = df_out.columns.droplevel()
freq_out = df_out[[df_out.columns[0],df_out.columns[2]]]

    
(freq_in,freq_out)

(              Pass To  FREQ M%
 0      Banton, Dalano      0.1
 1         Hauser, Sam      2.2
 2     Tillman, Xavier      0.1
 3   Pritchard, Payton      5.4
 4     Brissett, Oshae      1.0
 5     Mykhailiuk, Svi      0.2
 6        Kornet, Luke      0.2
 7      White, Derrick     27.2
 8       Tatum, Jayson     23.2
 9       Brown, Jaylen     19.3
 10      Holiday, Jrue     17.7
 11        Horford, Al      3.4,
             Pass From  FREQ R%
 0      Banton, Dalano      0.1
 1         Hauser, Sam      2.5
 2     Springer, Jaden      0.1
 3     Tillman, Xavier      0.2
 4      Stevens, Lamar      0.1
 5   Pritchard, Payton      7.1
 6     Brissett, Oshae      0.6
 7     Mykhailiuk, Svi      0.3
 8        Kornet, Luke      0.4
 9      White, Derrick     26.1
 10      Tatum, Jayson     24.0
 11      Brown, Jaylen     20.1
 12      Holiday, Jrue     15.4
 13        Horford, Al      3.0)

In [14]:
testDict = {'KristapsPorzingis' : (freq_in,freq_out)}

In [15]:
testDict['KristapsPorzingis']

(              Pass To  FREQ M%
 0      Banton, Dalano      0.1
 1         Hauser, Sam      2.2
 2     Tillman, Xavier      0.1
 3   Pritchard, Payton      5.4
 4     Brissett, Oshae      1.0
 5     Mykhailiuk, Svi      0.2
 6        Kornet, Luke      0.2
 7      White, Derrick     27.2
 8       Tatum, Jayson     23.2
 9       Brown, Jaylen     19.3
 10      Holiday, Jrue     17.7
 11        Horford, Al      3.4,
             Pass From  FREQ R%
 0      Banton, Dalano      0.1
 1         Hauser, Sam      2.5
 2     Springer, Jaden      0.1
 3     Tillman, Xavier      0.2
 4      Stevens, Lamar      0.1
 5   Pritchard, Payton      7.1
 6     Brissett, Oshae      0.6
 7     Mykhailiuk, Svi      0.3
 8        Kornet, Luke      0.4
 9      White, Derrick     26.1
 10      Tatum, Jayson     24.0
 11      Brown, Jaylen     20.1
 12      Holiday, Jrue     15.4
 13        Horford, Al      3.0)

In [16]:
csv_file = "testDict.csv"

# Open the CSV file in write mode
with open(csv_file, 'w', newline='') as file:
    # Create a CSV writer object
    writer = csv.writer(file)
    
    # Write the header row
    writer.writerow(['Team', 'Wins'])
    
    # Write each key-value pair as a row in the CSV file
    for team, wins in testDict.items():
        writer.writerow([team, wins])

print("CSV file has been saved as", csv_file)

CSV file has been saved as testDict.csv
